<a href="https://colab.research.google.com/github/claudiaxpreda/RQA-deBERTa/blob/main/Copy_of_SyntacticParserEnv.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Environment
!python -m pip install huggingface_hub
!pip install transformers
!pip install datasets
!pip install benepar

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.5 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.
  Preparing metadata (setup.py) ... done
  Created wheel for benepar: filename=benepar-0.2.0-py3-none-any.whl size=37626 sha256=426072b92daf1bc5d6ea417187db2212

In [ ]:
!python -m spacy download en_core_web_md

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 21.5 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
#Set variables to accees code
PATH_PREFIX = "/content/drive/MyDrive/"
WORK_DIR = PATH_PREFIX + 'CodeC/'
DATASET_DIR = WORK_DIR + 'Datasets/Fairytale/AnswersCandidates/'
DATSET_ORIGINAL = "readerbench/fairytale_qgen_contest"
ENV_PATH = WORK_DIR + 'environment.py'
#ENV_PATH = WORK_DIR + 'environment.py'

In [ ]:
#Modules to import
import os
import nltk
import sys
import re

import pandas as pd
from datasets import load_dataset
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
sys.path.append('/content/drive/MyDrive/CodeC')
print(os.getcwd())
print(WORK_DIR)

import environment

/content
/content/drive/MyDrive/CodeC/


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package benepar_en3 to /root/nltk_data...
[nltk_data]   Unzipping models/benepar_en3.zip.
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/benepar/parse_chart.py:169: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(
You are using the default legacy b

[unused1] - 1
[unused2] - 2
[unused3] - 3
[unused4] - 4
[CLS] - 101
[SEP] - 102
[PAD] - 0


In [ ]:
#Helpers
def prepare_input(dataset, slice_indx):
  context_list = dataset['context']
  df = pd.DataFrame(context_list, columns=['context'])
  df['context'] = df['context'].apply(lambda e : e.replace("\n", " ").strip())
  df['context'] = df['context'].apply(lambda e : e.replace("\r", " ").strip())
  # df['context'] = df['context'].apply(lambda e : re.sub(r'\s([?.!"](?:\s|$))', r'\1', e))
  # df['context'] = df['context'].apply(lambda e : e.rstrip())



  df.drop_duplicates(inplace=True)
  return df


In [ ]:
#Access datset HuggingFace
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) 
Token is valid (permission: read).
The token `Collab` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when push

In [ ]:
fairytale_dataset = load_dataset('readerbench/fairytale_qg_1024')

ft_train.csv:   0%|          | 0.00/9.50M [00:00<?, ?B/s]

ft_validation.csv:   0%|          | 0.00/1.07M [00:00<?, ?B/s]

ft_test.csv:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
df = fairytale_dataset['test']
df = prepare_input(df, '')

In [ ]:
def get_indexes(child, root):
  start = root.find(child);
  end = start + len(child)
  return (start, end)

def create_entry(full_context, seq, subtree):
  start_indx, end_indx = get_indexes(seq, full_context)
  return {'context': full_context, 'sequence': seq, 'indexes': [start_indx, end_indx], 'subtree': subtree}

def dfs(current_node, context, objects_list):
  for c in range(len(current_node.children)):
    child = current_node.children[c]
    objects_list.append(create_entry(context, child.text, child.subtree))
    dfs(child, context, objects_list)

  return;

def create_slice(dataset_list):
  slice_dict = []
  for entry in dataset_list:
    entry = entry.replace("\n", " ").strip()
    entry = entry.replace("\r", " ").strip()
    entry = " ".join(entry.split())
    env_obj =  environment.MyEnvironment(entry)
    dfs(env_obj.root, entry, slice_dict)
  return slice_dict

def creaate_dataset(dataset, slices):
  dataset_object = {}
  for sl in slices:
    slice_list = prepare_input(dataset, sl)
    slice_dict = create_slice(slice_list)
    dataset_object = {sl : slice_dict}
  return dataset_object

def create_df(sample):
  df = pd.DataFrame.from_dict(sample)
  df = df.loc[df.astype(str).drop_duplicates().index].reset_index(drop=True)
  return df

def write_to_file(dataset, slices, output_folder):
  dataset_object = creaate_dataset(dataset, slices)
  for sl in slices:
    df = create_df(dataset_object[sl])
    filename = output_folder + 'answer_candidates_new_' + sl +'.csv'
    df.to_csv(filename, index=False)

def generate_sequence_files():
  write_to_file(fairytale_dataset, ['test'], DATASET_DIR)
  write_to_file(fairytale_dataset, ['train'], DATASET_DIR)
  write_to_file(fairytale_dataset, ['validation'], DATASET_DIR)

In [ ]:
write_to_file(fairytale_dataset, ['validation'], DATASET_DIR)

Token indices sequence length is longer than the specified maximum sequence length for this model (525 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
import spacy
pos_all_tags = list(spacy.glossary.GLOSSARY.keys()) + ['ZZZ_SENTENCE', 'S', 'PRN', 'SQ', 'LST', 'SBARQ', 'ZZZ', 'NML', 'QP', 'WHNP', 'CONJP', 'SINV', 'WHADVP', 'WHPP', 'FRAG', 'UCP', 'WHADJP', 'RRC']
pos_all_tags.sort()
print(pos_all_tags)

['""', '#', '$', '$(', '$,', '$.', "''", ',', '-LRB-', '-RRB-', '.', ':', 'AD', 'ADD', 'ADJ', 'ADJA', 'ADJD', 'ADJP', 'ADP', 'ADV', 'ADVP', 'AFX', 'APPO', 'APPR', 'APPRART', 'APZR', 'ART', 'AS', 'AUX', 'BA', 'BES', 'CARD', 'CARDINAL', 'CC', 'CCONJ', 'CD', 'CONJ', 'CONJP', 'CS', 'DATE', 'DEC', 'DEG', 'DER', 'DET', 'DEV', 'DRV', 'DT', 'EOL', 'ETC', 'EVENT', 'EVT', 'EX', 'FAC', 'FACILITY', 'FM', 'FRAG', 'FW', 'GPE', 'GPE_LOC', 'GPE_ORG', 'GW', 'HVS', 'HYPH', 'IJ', 'IN', 'INTJ', 'ITJ', 'JJ', 'JJR', 'JJS', 'KOKOM', 'KON', 'KOUI', 'KOUS', 'LANGUAGE', 'LAW', 'LB', 'LC', 'LOC', 'LS', 'LST', 'M', 'MD', 'MISC', 'MONEY', 'MSP', 'NE', 'NFP', 'NIL', 'NML', 'NN', 'NNE', 'NNP', 'NNPS', 'NNS', 'NORP', 'NOUN', 'NP', 'NR', 'NT', 'NUM', 'OD', 'ON', 'ORDINAL', 'ORG', 'P', 'PART', 'PAV', 'PDAT', 'PDS', 'PDT', 'PER', 'PERCENT', 'PERSON', 'PIAT', 'PIDAT', 'PIS', 'PN', 'PNP', 'POS', 'PP', 'PPER', 'PPOSAT', 'PPOSS', 'PRELAT', 'PRELS', 'PRF', 'PRN', 'PROAV', 'PROD', 'PRODUCT', 'PRON', 'PROPN', 'PRP', 'PRP$', 'P

In [ ]:
# Test cell 1
sample = prepare_input(fairytale_dataset_og, 'test')
environment_sample = environment.MyEnvironment(sample[0])
pos = environment_sample.root.pos

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
/usr/local/lib/python3.10/dist-packages/torch/distributions/distribution.py:56: UserWarning: <class 'torch_struct.distributions.TreeCRF'> does not define `arg_constraints`. Please set `arg_constraints = {}` or initialize the distribution with `validate_args=False` to turn off validation.
  warnings.warn(


In [ ]:
# Test cell 2
print(environment_sample.root)
# print (pos_all_tags[154])
# print (pos_all_tags.index('ZZZ_SENTENCE'))

TOK_IDS: [101, 1, 3, 1517, 1852, 170, 1159, 1175, 1108, 170, 2226, 1150, 1355, 5275, 1154, 1103, 1362, 1105, 21616, 1174, 1171, 170, 2712, 5746, 119, 4, 3, 1105, 1170, 1152, 1125, 1151, 1597, 170, 1229, 5540, 1522, 1172, 170, 1376, 1797, 119, 4, 3, 1173, 1175, 1108, 1632, 1231, 5077, 27989, 2118, 1107, 1103, 1331, 1105, 2032, 1103, 1583, 117, 1111, 1103, 1234, 5608, 1147, 2226, 1155, 1115, 1108, 1363, 117, 1290, 1119, 1108, 1912, 1105, 1198, 119, 4, 3, 1229, 1103, 2027, 3191, 1107, 1157, 26822, 117, 170, 4020, 118, 1702, 1385, 1590, 2242, 1103, 1395, 117, 1105, 1185, 1141, 1450, 1150, 1131, 1108, 4040, 1165, 2093, 1131, 1338, 119, 4, 3, 1103, 1385, 1590, 2910, 170, 8062, 1166, 1103, 2027, 117, 1105, 1163, 1115, 1131, 1538, 1136, 1129, 2148, 1149, 1223, 1103, 1501, 3901, 1235, 1131, 1127, 1554, 5147, 1201, 1104, 1425, 117, 1290, 4303, 1103, 3231, 28000, 1156, 21616, 1123, 119, 4, 3, 1165, 1103, 2226, 1767, 1142, 1119, 1261, 1123, 1734, 1106, 1762, 117, 1105, 6310, 5292, 1106, 2824, 1166

In [ ]:
# Test cell 1
print(environment_sample.root.children[1])

start, end = get_indexes(environment_sample.root.children[1].text, environment_sample.root.text)

print(environment_sample.root.text[start:end])
print(environment_sample.root.children[1].subtree)

TOK_IDS: [101, 1, 3, 1135, 2171, 1115, 1131, 2204, 5178, 4, 117, 3, 1105, 4, 3, 1165, 1131, 1464, 1115, 1131, 1538, 1770, 2939, 117, 1131, 1850, 1111, 1103, 1624, 117, 1105, 1163, 117, 112, 1409, 1128, 1328, 1106, 5390, 1170, 1139, 1473, 117, 1294, 1185, 1141, 5746, 4895, 1131, 1110, 1198, 1112, 2712, 1112, 146, 1821, 117, 1105, 1144, 1198, 1216, 5404, 1716, 1112, 146, 1138, 4, 119, 2, 102] 
 ACTIONS: [[1, 68], [2, 9], [11, 13], [14, 66], None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None] 
 CHILDREN: 3 
 TEXT: It happened that she fell ill, and when she felt that she must soon die, she sent for the King, and said, 'If you want to marry after my death, make no one queen unless she is just as beautiful as I am, and has just such golden hair as I have. 
 SUBTREE: 93
It happened that she fell ill, and when she felt that she must soon die, she sent for the King, and said, 'If you want to